# Initialize the functions

In [3]:
from functions import clean_txt_file

# Set the file locations and run the functions

In [4]:
input_file = "./path/to/input_file.txt"
output_file = "./path/to/desired_output_location.txt"

# system message for AI
system_message = """You are cleaning a transcript. Your task is to:
            1. Fix any obvious transcription errors
            2. Maintain the speaking style and informality while making the text more readable and continuous
            3. Keep all original meaning intact
            4. Use the previous context to maintain consistency
            5. In some cases, the text from a speaker might get added to the next speaker. Ensure that the diarization is done properly. Combine the text if needed. But apply extra caution while combining multiple blocks into one
            6. Remove fillers like "um", "uh", "you know", repeated "okays" that are redundant.
            Background to the conversation: State the context of the conversation, if it is not clear from the text. For example, "The speaker is explaining the process of photosynthesis to a group of students."
            """

clean_txt_file(input_file, output_file, chunk_size=10, system_message=system_message)

Processing chunk 1/12
Processing chunk 2/12
Processing chunk 3/12
Processing chunk 4/12
Processing chunk 5/12
Processing chunk 6/12
Processing chunk 7/12
Processing chunk 8/12
Processing chunk 9/12
Processing chunk 10/12
Processing chunk 11/12
Processing chunk 12/12
Cleaned transcript saved to ./data/clean/shivakumara_clean.txt
